In [194]:
import json
import pandas as pd
import numpy as np
import os
from IPython.display import display
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
import google.generativeai as genai

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY') 

genai.configure(api_key=GOOGLE_API_KEY)

In [121]:
vg_path_1 = "/mmfs1/gscratch/krishna/clip_hn/datasets/vg/VG_100K"
vg_path_2 = "/mmfs1/gscratch/krishna/clip_hn/datasets/vg/VG_100K_2"

In [122]:
vg_path = "/gscratch/krishna/shared_data/datasets/vg" 

In [190]:
def tag(img, objs):
    W, H = img.size
    img1 = img.copy()
    draw = ImageDraw.Draw(img1)
    for i, obj in enumerate(objs):
        box = obj["bbox"]
        draw.rectangle(box, outline="red", width=6)
        x1, y1, x2, y2 = box
        label = obj["name"]

        font = ImageFont.load_default()
        font_box = font.getbbox(label)
        text_width = font_box[2] - font_box[0]
        text_height = font_box[3] - font_box[1]

        if x1 + text_width > W:
            x1 = x1 - text_width
        if y1 + text_height > H:
            y1 = y1 - text_height

        draw.rectangle((x1, y1 - text_height, x1 + text_width, y1), fill="red")
        draw.text((x1, y1 - text_height), label, fill="white", font=font)
    return img1

In [124]:
# load vg scene graphs as a df 
sg = json.load(open(os.path.join(vg_path, "scene_graphs.json")))
sg_df = pd.DataFrame.from_records(sg)
sg_df.head()

,relationships,image_id,objects
0,"[{'synsets': ['along.r.01'], 'predicate': 'ON'...",1,"[{'synsets': ['clock.n.01'], 'h': 339, 'object..."
1,"[{'synsets': ['wear.v.01'], 'predicate': 'wear...",2,"[{'synsets': [], 'h': 103, 'object_id': 5069, ..."
2,"[{'synsets': ['in.r.01'], 'predicate': 'in fro...",3,"[{'synsets': [], 'h': 79, 'object_id': 5091, '..."
3,"[{'synsets': ['have.v.01'], 'predicate': 'has'...",4,"[{'synsets': ['curtain.n.01'], 'h': 300, 'obje..."
4,"[{'synsets': ['along.r.01'], 'predicate': 'ON'...",5,"[{'synsets': ['floor.n.01'], 'h': 108, 'object..."


In [186]:
# for idx in range(5):
#     objects = sg_df['objects'].iloc[idx]
#     for obj in objects:
#         if len(obj['names']) == 1:
#             print(obj['names'][0])
#         if 'attributes' in obj:
#             print(obj['attributes'])

In [187]:
sg_df['objects'].iloc[0]

[{'synsets': ['clock.n.01'],
  'h': 339,
  'object_id': 1058498,
  'names': ['clock'],
  'w': 79,
  'attributes': ['green', 'tall'],
  'y': 91,
  'x': 421},
 {'synsets': ['street.n.01'],
  'h': 262,
  'object_id': 5046,
  'names': ['street'],
  'w': 714,
  'attributes': ['sidewalk'],
  'y': 328,
  'x': 77},
 {'synsets': ['shade.n.01'],
  'h': 192,
  'object_id': 5045,
  'names': ['shade'],
  'w': 274,
  'y': 338,
  'x': 119},
 {'synsets': ['man.n.01'],
  'h': 262,
  'object_id': 1058529,
  'names': ['man'],
  'w': 60,
  'y': 249,
  'x': 238},
 {'synsets': ['gym_shoe.n.01'],
  'h': 26,
  'object_id': 5048,
  'names': ['sneakers'],
  'w': 52,
  'attributes': ['grey'],
  'y': 489,
  'x': 243},
 {'synsets': ['headlight.n.01'],
  'h': 15,
  'object_id': 5050,
  'names': ['headlight'],
  'w': 23,
  'attributes': ['off'],
  'y': 366,
  'x': 514},
 {'synsets': ['car.n.01'],
  'h': 98,
  'object_id': 5049,
  'names': ['car'],
  'w': 74,
  'y': 315,
  'x': 479},
 {'synsets': ['bicycle.n.01'],
  

In [179]:
sg_df['relationships'].iloc[0]

[{'synsets': ['along.r.01'],
  'predicate': 'ON',
  'relationship_id': 15927,
  'object_id': 5046,
  'subject_id': 5045},
 {'synsets': ['wear.v.01'],
  'predicate': 'wears',
  'relationship_id': 15928,
  'object_id': 5048,
  'subject_id': 1058529},
 {'synsets': ['have.v.01'],
  'predicate': 'has',
  'relationship_id': 15929,
  'object_id': 5050,
  'subject_id': 5049},
 {'synsets': ['along.r.01'],
  'predicate': 'ON',
  'relationship_id': 15930,
  'object_id': 1058508,
  'subject_id': 1058507},
 {'synsets': ['along.r.01'],
  'predicate': 'ON',
  'relationship_id': 15931,
  'object_id': 1058534,
  'subject_id': 5055},
 {'synsets': ['have.v.01'],
  'predicate': 'has',
  'relationship_id': 15932,
  'object_id': 1058511,
  'subject_id': 1058529},
 {'synsets': ['next.r.01'],
  'predicate': 'next to',
  'relationship_id': 15933,
  'object_id': 1058539,
  'subject_id': 1058534},
 {'synsets': ['have.v.01'],
  'predicate': 'has',
  'relationship_id': 15934,
  'object_id': 5060,
  'subject_id': 1

In [171]:
# sg_df['relationships_length'] = sg_df.apply(lambda row: len(row['relationships']), axis=1)
# sg_df['relationships_length'].mean()

In [145]:
np.random.seed(42)
random_ids = np.random.choice(sg_df['image_id'], 100)
print("image ids:", list(random_ids))

image ids: [2406384, 861, 2413814, 2342508, 2365475, 498069, 2336685, 2384008, 2331329, 2376714, 2359854, 2406152, 2379926, 2352561, 2355085, 770, 2360408, 2357027, 2354973, 2351785, 150259, 2414288, 2412432, 2335926, 2366717, 2333624, 2392886, 2347635, 2325561, 2416838, 2396086, 2334491, 2403624, 2748, 2361014, 2354132, 2334308, 2385496, 2352339, 2363380, 2352999, 2389920, 2410992, 2350606, 3891, 2379376, 2322149, 2339138, 2331524, 2412733, 1591952, 2345506, 2377915, 2342788, 2397915, 2351596, 2398344, 2372115, 2404959, 2416370, 2339097, 2385340, 2326554, 2352671, 2350201, 2329262, 2402546, 2353258, 2342119, 2340283, 2367455, 2380280, 1592852, 2375013, 2347137, 2348395, 2354162, 2384142, 2326528, 2318997, 2388820, 2410848, 2323834, 2380654, 2327257, 1017, 2328932, 2364737, 2328907, 2398583, 2397497, 2345407, 2336248, 1592428, 2332056, 2410166, 2356243, 2332084, 2382024, 1160221]


In [167]:
df_ids = sg_df[sg_df['image_id'].isin(random_ids)].index
print("df ids:", list(df_ids))

df ids: [769, 860, 1016, 2747, 3890, 5311, 6265, 8571, 8792, 9268, 9692, 10627, 11394, 11534, 12185, 15795, 16023, 17159, 18431, 19457, 23247, 23483, 23897, 24300, 25658, 28693, 31551, 32606, 35773, 35920, 37065, 37194, 39099, 40397, 40757, 41090, 41606, 43001, 44131, 45758, 48555, 52995, 53707, 54886, 55591, 56886, 59150, 59735, 60263, 62955, 63704, 64820, 64925, 65697, 65725, 66557, 66803, 67121, 67221, 67435, 67969, 68148, 69092, 69479, 71211, 71932, 72409, 73969, 74065, 76552, 76820, 77189, 78953, 80038, 80077, 82386, 82798, 83104, 84478, 84654, 85305, 86779, 86807, 87313, 87498, 89475, 89789, 89812, 91387, 92067, 92093, 93016, 94663, 96276, 99299, 103355, 103694, 104724, 106530, 106970]


In [7]:
def get_vg_img_from_idx(idx):
    img_path = os.path.join(vg_path_1, f"{idx}.jpg")
    if not os.path.exists(img_path):
        img_path = os.path.join(vg_path_2, f"{idx}.jpg")
    img = Image.open(img_path)
    return img

In [8]:
def display_after_resize(img, resize=True):
    if resize:
        original_width, original_height = img.size
    
        new_height = 512  # or whatever new height you want
        aspect_ratio = original_height / original_width
        new_width = int(new_height * aspect_ratio)
        img = img.resize((new_height, new_width))
    
    display(img)

In [264]:
def get_obj_dict_list(img, objects, k=0, r=0.01):
    list_of_obj_dict = []
    id2name = {}
    name2bbox = {}
    for i, obj in enumerate(objects):
        if len(obj['names']) > 1: # consider only the objects with one name
            print(obj['names'])
            continue
        if 'attributes' not in obj or len(obj['attributes']) == 0:
            continue
        obj_area = obj['w'] * obj['h']
        img_area = np.prod(img.size)
        obj_area_ratio = obj_area / img_area
        if obj_area_ratio < r: # consider only objects with an area ration >= r
            continue
    
        bbox = [obj['x'], obj['y'], obj['x'] + obj['w'], obj['y'] + obj['h']]
        obj_dict = {'object_id': obj['object_id'], 'name': obj['names'][0], 'bbox': bbox, 'area': obj_area}
        id2name[obj['object_id']] = obj['names'][0]
        name2bbox[obj['names'][0]] = bbox
        list_of_obj_dict.append(obj_dict)
        
    if k > 0: # if specified, select only the top k objects with largest areas
        list_of_obj_dict = sorted(list_of_obj_dict, key=lambda x: -x['area'])[:k] 
    return list_of_obj_dict, id2name, name2bbox

In [265]:
def get_obj_pairs_in_relationships(relationships):
    pairs = set()
    for rlt in relationships:
        obj1 = rlt['subject_id']
        obj2 = rlt['object_id']
        pair = (obj1, obj2)
        pairs.update([pair])
    return list(pairs)

In [266]:
def get_response(prompt, image):
  model = genai.GenerativeModel('gemini-pro-vision')
  response = model.generate_content([prompt, image], stream=False)
  response.resolve()
  return response.text

In [154]:
def get_attribute_prompt(object):
    prompt = "What adjectives would you use to describe the specified object?\n"
    prompt += "Please answer this question with a comma-separated list of adjectives like the examples below.\n"
    prompt += "Examples:\n"
    prompt += "cat: white, sitting, small\n"
    prompt += "car: black, parked\n"
    prompt += "sidewalk: brick, red\n"
    prompt += "{object}:"
    return prompt.format(object=object)

In [215]:
def get_relationship_prompt(object1, object2):
    prompt = "What are the relationships of {object1} to {object2}?\n"
    prompt += "Please answer this question with a comma-separated list like the examples below.\n"
    prompt += "Examples:\n"
    prompt += "cat -> carpet: on, above\n"
    prompt += "car -> tree: near\n"
    prompt += "bike -> sidewalk: parked on\n"
    prompt += "{object1} -> {object2}:\n"
    # prompt += "{object2} -> {object1}:\n"
    return prompt.format(object1=object1, object2=object2)

In [ ]:
# tag and display images with all bounding boxes
# outputs = {}
# for idx in range(100, 110):
#     row = sg_df.iloc[idx, :]
#     img_idx = row['image_id']
#     print(img_idx)
#     img = get_vg_img_from_idx(img_idx)
#     objects = row['objects']
#     obj_dict_list = get_obj_dict_list(img, objects, k=10, r=0)
#     print(len(obj_dict_list))
#     tagged_img = tag(img, obj_dict_list)
#     display_after_resize(tagged_img)

In [219]:
def parse_response_into_relationships(response, obj1_name, obj2_name):
    start = max(response.find(f'{obj1_name} -> {obj2_name}:'), 0) # find the relevant start or start from the beginning
    newline_pos = response[start:].find('\n')
    end = newline_pos if newline_pos > -1 else len(response)

    split_res = response[start:end].split(':')
    if len(split_res) >= 2:
    
        rlts = split_res[1].strip().split(',')
    else:
        rlts = response[start:end].strip().split(',')
    return rlts

In [261]:
K = 5 # select only the top k objects with largest areas
output_file = 'output_defined.csv'

In [ ]:
# generate attributes and relationships given images and bounding boxes of some objects 
# outputs = {}

for i, idx in enumerate(df_ids):
    try:
        row = sg_df.iloc[idx, :]
        img_idx = row['image_id']
        output = {'image_id': img_idx, "objects": [], "relationships": []}
        img = get_vg_img_from_idx(img_idx)
        objects = row['objects']
        obj_dict_list, id2name, name2bbox = get_obj_dict_list(img, objects, k=K, r=0)
        num_objs = len(obj_dict_list)
        # print(num_objs)
        # print(row['relationships'])
        obj_pairs = get_obj_pairs_in_relationships(row['relationships'])
        print(len(obj_pairs))
    
        # generate attributes
        for obj in obj_dict_list:
            tagged_img = tag(img, [obj])
            # display_after_resize(tagged_img)
            prompt = get_attribute_prompt(obj['name'])
            # print(prompt)
            response = get_response(prompt, tagged_img)
            print(response)
            obj['attributes'] = response.strip().split(',')
            output['objects'].append(obj)
    
        # generate relationships 
        for pair in obj_pairs:
            if pair[0] in id2name and pair[1] in id2name:
                obj1_name = id2name[pair[0]]
                obj2_name = id2name[pair[1]]
                obj1 = {'object_id': pair[0], 'name': obj1_name, 'bbox': name2bbox[obj1_name]}
                obj2 = {'object_id': pair[1], 'name': obj2_name, 'bbox': name2bbox[obj2_name]}
                tagged_img = tag(img, [obj1, obj2])
                # display_after_resize(tagged_img)
                prompt = get_relationship_prompt(obj1['name'], obj2['name'])
                # print(prompt)
                response = get_response(prompt, tagged_img)
                print(response)
                
                rlts = parse_response_into_relationships(response, obj1_name, obj2_name)
                
                rlt_obj = {'subject_id': obj1['object_id'], 'object_id': obj2['object_id'],  'relationships': rlts}
                output['relationships'].append(rlt_obj)
               
            else:
                print(pair)
        pd.DataFrame([output]).to_csv(output_file, mode='a',index=False,header=(i==0))
    except Exception as e:
        print(e)

8
 red, brick
 white, van, mercedes
 white, parked
 parked, gray, small
 gray, parked
(1637140, 1637133)
(1637154, 3686356)
(1637136, 1637138)
(1637153, 1637154)
(1637140, 1637132)
(1637139, 1637140)
(1637136, 1637137)
(1637129, 1637130)
['person', 'man']
['person', 'man']
['person', 'man']
39
 busy, crowded, narrow, covered
 brown, square, ceramic
 clear, plastic, slanted
 pink, fluffy, delicious
 green, open
(1030017, 1030027)
(1030054, 1030055)
(1030022, 1030023)
(1030020, 1030056)
(1030020, 1030059)
(1030032, 1030033)
(1030041, 1030042)
(1030017, 1030014)
(1030061, 1030020)
(1030019, 1030014)
(1030065, 1030066)
(1030030, 1030029)
(1030028, 1030029)
(1030042, 1030052)
(1030020, 1030058)
(1030017, 1030038)
(1030024, 1030025)
(1030020, 1030021)
(1030045, 1030020)
(1030024, 1030049)
(1030039, 1030043)
(1030039, 1030040)
(1030034, 1030035)
(1030018, 1030014)
(1030020, 1030042)
(1030044, 1030045)
(1030067, 1030035)
(1030036, 1030035)
(1030018, 1030026)
(1030018, 1030032)
(1030020, 103006

In [228]:
print(outputs)

{770: {'objects': [], 'relationships': [{'subject_id': 1637140, 'object_id': 1637133, 'relationships': ['on', ' above']}, {'subject_id': 1637154, 'object_id': 3686356, 'relationships': ['in', ' part of']}, {'subject_id': 1637136, 'object_id': 1637138, 'relationships': ['near', ' parallel']}, {'subject_id': 1637153, 'object_id': 1637154, 'relationships': ['on', ' crossing', ' stopped at', ' driving on']}, {'subject_id': 1637140, 'object_id': 1637132, 'relationships': ['on', ' above']}, {'subject_id': 1637139, 'object_id': 1637140, 'relationships': ['part of']}, {'subject_id': 1637136, 'object_id': 1637137, 'relationships': ['near', ' crossing']}, {'subject_id': 1637129, 'object_id': 1637130, 'relationships': ['crossing', ' on']}]}, 861: {'objects': [], 'relationships': [{'subject_id': 1030054, 'object_id': 1030055, 'relationships': ['connected to', ' has']}, {'subject_id': 1030022, 'object_id': 1030023, 'relationships': ['above']}, {'subject_id': 1030020, 'object_id': 1030056, 'relation

In [140]:
# structure
# {
#     image_id:
#      {
#         "objects": [{'object_id': 000, 'name': 'cat', 'attributes': ['white', 'small']}, {'object_id': 001, 'name': 'carpet', 'attributes': ['grey', 'fluffy']}]
#         "relationships": [{'subject_id': 000, 'object_id': 001,  'relationships': ['on']}]
#      }
# }